# Librerias

In [1]:
from google.colab import drive  #Para enlazar a Drive
import pandas as pd             #Para manejar la base de datos
import glob, os                 #Para leer archivos de drive
import datetime                 #Para manejar fechas
import numpy as np
import time

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

#drive.mount('/content/drive')

In [2]:
path = 'drive/MyDrive/IPC Bienes/Datos (Scraper)/'
productos = ['TV']
'''
productos = ['Celulares',
             'TV',
             'Impresora',
             'Lavadora - Secadora',
             'Computador', 
             'Refrigerador'
             ]
'''
db_productos = {}
for producto in productos:
  db={}
  i = 1
  for filename in glob.glob(os.path.join(path+producto, '*.csv')):
    #print(filename)
    x = pd.read_csv(filename, encoding='utf-8')
    x = x[~x.Modelo.str.contains(" \+")]
    try: x = x.rename({'Precio_Normal': 'Precio Normal'}, axis=1)
    except: pass
    db[i] = x
    i+=1
  db_productos[producto] = pd.concat(db.values(), ignore_index=True)
  #df.to_csv('drive/MyDrive/IPC Bienes/'+producto+'.csv', index=False)

In [3]:
df = db_productos['TV']

In [4]:
#Precio a int
try:
  df['Precio Normal'] = df['Precio Normal'].replace('[\$,]','',regex=True).replace('[\.,]','',regex=True).replace('[\",]','',regex=True).astype(int)
except:
  for j, row in df.iterrows():
    valor = row['Precio Normal']
    try:
      df.at[j, 'Precio Normal'] = valor.replace('[\$,]','',regex=True).replace('[\.,]','',regex=True).replace('[\",]','',regex=True).astype(int)
    except:
      df.at[j, 'Precio Normal'] = np.nan

df = df.loc[df['Tienda'].isin(['Paris', 'Falabella', 'Ripley', 'Linio Chile', 'Hites', 'La Polar']), :]
#Agrupamos por Año-Mes
df.loc[:,'Fecha'] = pd.to_datetime(df['Fecha'])
df.loc[:,'Fecha'] = df['Fecha'].dt.strftime('%Y-%m')
VAR = df.groupby(['Tienda', 'Marca', 'Modelo', 'Pulgadas', 'Resolucion', 'Fecha'], as_index=False).mean()
VAR.sort_values(by=['Tienda', 'Marca', 'Modelo', 'Pulgadas', 'Resolucion', 'Fecha'], inplace=True)
VAR.reset_index(inplace=True, drop = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
VAR.head()

,Tienda,Marca,Modelo,Pulgadas,Resolucion,Fecha,Precio Normal
0,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-04,189990.000000
1,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-05,181418.571429
2,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-06,181528.461538
3,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-07,179990.000000
4,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-08,169990.000000


In [6]:
VAR['IVE_tve'] = VAR.sort_values('Fecha')['Precio Normal'].div(VAR.groupby(['Tienda', 'Marca', 'Modelo', 'Pulgadas', 'Resolucion'])['Precio Normal'].shift(1)) *100

In [7]:
VAR.head(10)

,Tienda,Marca,Modelo,Pulgadas,Resolucion,Fecha,Precio Normal,IVE_tve
0,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-04,189990.000000,NaN
1,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-05,181418.571429,95.488484
2,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-06,181528.461538,100.060573
3,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-07,179990.000000,99.152496
4,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-08,169990.000000,94.444136
5,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-09,179990.000000,105.882699
6,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-10,179990.000000,100.000000
7,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-11,173323.333333,96.296091
8,Falabella,AOC,32S5295,"32.0""",HD (1366x768),2021-12,157490.000000,90.864858
9,Falabella,AOC,32S5305,"32.0""",HD (1366x768),2021-12,169990.000000,NaN


In [16]:
#VAR.replace(0, np.nan, inplace=True)
VAR.dropna(inplace=True)

IVAR = VAR.groupby(['Marca', 'Modelo', 'Pulgadas', 'Resolucion', 'Fecha'], as_index=False).agg({'IVE_tve':['prod', 'count']})
IVAR.columns = ['Marca', 'Modelo', 'Pulgadas', 'Resolucion', 'Fecha', 'IVAR_tv', 'IVAR_tv_count']
#IVAR_count = VAR.groupby(['Marca', 'Modelo', 'Pulgadas', 'Resolucion', 'Fecha'], as_index=False)['IVE_tve'].count()

#IVAR = IVAR.rename({'IVE_tve': 'IVAR_tv'}, axis=1)
IVAR.loc[:, 'IVAR_tv'] = IVAR['IVAR_tv']**(1/IVAR['IVAR_tv_count'])

In [17]:
IVAR.head()

,Marca,Modelo,Pulgadas,Resolucion,Fecha,IVAR_tv,IVAR_tv_count
0,AOC,32S5295,"32.0""",HD (1366x768),2021-05,100.097015,5
1,AOC,32S5295,"32.0""",HD (1366x768),2021-06,98.806791,6
2,AOC,32S5295,"32.0""",HD (1366x768),2021-07,100.943675,6
3,AOC,32S5295,"32.0""",HD (1366x768),2021-08,95.735946,6
4,AOC,32S5295,"32.0""",HD (1366x768),2021-09,105.243657,6


In [ ]:
#IVAR.replace(0, np.nan, inplace=True)
IVAR.dropna(inplace=True)

IPRO_count = IVAR.groupby(['Fecha'], as_index=False)['IVAR_tv'].count()
IVAR = pd.merge(IVAR, IPRO_count, on='Fecha')
IVAR.loc[:, 'IPRO_tv'] = IVAR['IVAR_tv_x']**(1/IVAR['IVAR_tv_y'])

IPRO = IVAR.groupby(['Fecha'], as_index=False)['IPRO_tv'].prod()

IPRO = IPRO.rename({'IPRO_tv' : 'IPRO_t'}, axis=1)

In [ ]:
VAR[(VAR['Modelo'] == '32S5295') & (VAR['Fecha'] == '2021-05')].prod()

Precio Normal    1.839796e+26
IVE_tve          1.004860e+10
dtype: float64

In [ ]:
IPRO['IPRO_t'].diff()

0         NaN
1    1.210195
2   -0.625454
3    0.093604
4    0.882871
5   -1.212823
6   -3.678748
7   -1.389768
Name: IPRO_t, dtype: float64

In [ ]:
pd.merge(IVAR, IPRO_count, on='Fecha')

,Marca,Modelo,Pulgadas,Resolucion,Fecha,IVAR_tv_x,IVAR_tv_y,IPRO_tv,IVAR_tv
0,AOC,32S5295,"32.0""",HD (1366x768),2021-05,100.097015,226,1.020590,226
1,AOC,43S5295,"43.0""",HD (1366x768),2021-05,95.332359,226,1.020370,226
2,AOC,43S5305,"43.0""",Full HD (1920x1080),2021-05,93.288259,226,1.020272,226
3,AOC,50U6295,"50.0""",4k Ultra HD (3840x2160),2021-05,97.532477,226,1.020473,226
4,Aiwa,AW32B4SM,"32.0""",HD (1366x768),2021-05,102.494286,226,1.020697,226
...,...,...,...,...,...,...,...,...,...
1953,TCL,55P725,"55.0""",4k Ultra HD (3840x2160),2021-12,93.167561,226,1.020266,226
1954,TCL,65P725,"65.0""",4k Ultra HD (3840x2160),2021-12,93.539382,226,1.020284,226
1955,TCL,70P615,"70.0""",4k Ultra HD (3840x2160),2021-12,82.914565,226,1.019740,226
1956,Winia,L32V750BAS,"32.0""",HD (1366x768),2021-12,94.014754,226,1.020307,226
